# Analyze the pilot set to inform current set, then order

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/projects/crispy_shifty/projects/crispy_shifties
running on node: dig67


### Set working directory to the root of the crispy_shifty repo

In [2]:
os.chdir("/projects/crispy_shifty")

### First need to fix the `final_order_scores.csv` 
The one with the `_old` suffix has correct design naming, but doesn't include new chB sequences

In [3]:
new_final_scores = os.path.join(
    os.getcwd(), f"projects/crispy_shifties/09_filter_and_order/final_order_scores.csv"
)
old_final_scores = os.path.join(
    os.getcwd(),
    f"projects/crispy_shifties/09_filter_and_order/final_order_scores_old.csv",
)
new_scores_df = pd.read_csv(new_final_scores, index_col="pdb_path")
old_scores_df = pd.read_csv(old_final_scores, index_col="pdb_path")

### Check the differences in columns
As expected, the new df has the 2nd resurfacing and the old has the 1st.

In [4]:
print(set(new_scores_df.columns) - set(old_scores_df.columns))
print(set(old_scores_df.columns) - set(new_scores_df.columns))

{'B_final_seq_r2'}
{'B_final_seq_r1'}


### Check the differences in column values
The new df messed up the id due to iteration over an unsorted set

In [5]:
cols = list(old_scores_df.columns)

cols.remove("mean_pae_interaction")
cols.remove("B_final_seq_r1")

cols = list(sorted(cols))

for col in tqdm(cols):
    if dict(old_scores_df[col]) != dict(new_scores_df[col]):
        print(col)
    else:
        pass

  0%|          | 0/149 [00:00<?, ?it/s]

id


### Merge the two dfs
Keep indexing and values from old_scores_df, add 2nd resurfacing and incorrectly labeled ID

In [6]:
update_df = pd.DataFrame(new_scores_df[["id", "B_final_seq_r2"]])
update_df["incorrect_id"] = update_df["id"]
update_df.drop("id", axis=1, inplace=True)
fixed_df = pd.concat([old_scores_df, update_df], axis=1)
fixed_df.head()

,A_final_seq,A_sap,A_score_per_res,A_total_score,A_wnm_all,A_wnm_helix,B_final_seq,B_sap,B_score_per_res,B_total_score,...,delta_sap_XY,delta_sap_AC,delta_total_score_XY,delta_total_score_AC,A_charge,B_charge,id,B_final_seq_r1,B_final_seq_r2,incorrect_id
pdb_path,,,,,,,,,,,,,,,,,,,,,
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0006/09_filter_and_order_e9b844167bf84dd2ac7cde5045dc43b1.pdb.bz2,DEEVQEAVERAEELREEAEELIKKARKTGDPELLRKALEALEEAVR...,70.064735,-1.903443,-437.791779,0.485354,0.086336,DEKEIKELAKRVAEKIREAFKIAAEGIKKGLEKK,40.034809,-1.143339,-38.873539,...,-3.854572,-31.276978,-517.192989,-96.144745,-12.039856,2.771885,cs_200,IKELAEEVAEKIREAFEIAAEGIEKGLEKR,DIKELAEEVAEKIREAFEIAAQGIEKGL,cs_219
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0007/09_filter_and_order_739f0d7fe9bd45f986942789620a6f94.pdb.bz2,DEEVQEAVERAEELREEAEELIKKARKTGDPELLRKALEALKEAVR...,57.506268,-2.112674,-485.915070,1.525385,0.076361,VEKEILELAKEVAEEIKEAIKIAAEGIRKGLEKK,49.365849,-1.043685,-35.485302,...,-9.132225,-20.905426,-482.052235,-24.410553,-14.027188,-1.247720,cs_201,SILELAEEVAEEIKEAIRIAAKGIREGLKE,SILELAEEVAEEIKEAIQIAAEGIRRGL,cs_220
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0007/09_filter_and_order_bd28e57868d440cdb4173b0c83f783f5.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLRELARALVEEAEAVA...,66.934334,-2.067249,-475.467133,0.617158,0.165406,EEEELLRELAELLVRVGRELLEMVEEQVKMIEERK,47.761921,-0.916358,-32.072525,...,-12.801592,-36.269396,-461.256271,-72.235199,-13.394478,-6.134295,cs_202,LLEELAELLVKVGRELLKMVQEQVQMIEER,LLEELAELLVEVGRELLKMVEEQVQMIK,cs_213
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_d07a25241ccd4c968d9c959e563c6ecb.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLEELAEALIEEAKAVA...,64.631165,-2.199711,-505.933533,0.491056,0.176779,AVREKLRELVELLVEVGRRLERVVRRAAERVKRRK,49.346237,-0.892982,-31.254385,...,-20.927460,-29.915047,-556.007822,-38.304382,-28.370214,6.809640,cs_203,VREKLRELVELLVEVGRRLEEVVKRAAERV,KLRELVELLVEVGRRLEEVVERAAERVE,cs_214
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_b06794938c094af0a9e8bf480b919640.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLEELAEALIEEAKAVA...,73.201729,-2.277482,-523.820801,0.348891,0.083955,AVREKLRELVELLVRVGERLRRVAERAAELVKRRR,41.399914,-0.937563,-32.814716,...,-11.018291,-30.554176,-594.738811,-18.317566,-22.385265,5.810639,cs_204,VEEKLRELVELLVHVGERLREVAERAAELV,KLRELVELLVEVGERLREVAERAAELVE,cs_215


### Add a note indicating which things were incorrectly named in 20220413 order
also dump fixed scores

In [7]:
note_df = pd.DataFrame(fixed_df[["id", "incorrect_id"]])

output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/09_filter")

if not os.path.exists(os.path.join(output_path, "fixed_final_order_scores.csv")):
    fixed_df.to_csv(
        os.path.join(output_path, "fixed_final_order_scores.csv"),
        index_label="pdb_path",
    )
if not os.path.exists(os.path.join(output_path, "incorrect_ids.csv")):
    note_df.to_csv(
        os.path.join(output_path, "incorrect_ids.csv"), index_label="pdb_path"
    )

### Load the current set

In [8]:
scores_path = os.path.join(
    os.getcwd(), f"projects/crispy_shifties/09_filter/scores.csv"
)

scores_df = pd.read_csv(scores_path, index_col="Unnamed: 0")

### Load the pilot set

In [9]:
scores_path = os.path.join(
    os.getcwd(), f"projects/crispy_shifties/09_filter/fixed_final_order_scores.csv"
)
pilot_df = pd.read_csv(scores_path, index_col="pdb_path")

### Setup for plotting

In [10]:
sns.set(
    context="talk",
    font_scale=1,  # make the font larger; default is pretty small
    style="ticks",  # make the background white with black lines
    palette="colorblind",  # a color palette that is colorblind friendly!
)

### Remove score terms we don't care about

In [11]:
from crispy_shifty.protocols.design import beta_nov16_terms

sample_df = pilot_df[
    [term for term in pilot_df.columns if term not in beta_nov16_terms]
]
print(len(sample_df))
print(list(sample_df.columns))

24
['A_final_seq', 'A_sap', 'A_score_per_res', 'A_total_score', 'A_wnm_all', 'A_wnm_helix', 'B_final_seq', 'B_sap', 'B_score_per_res', 'B_total_score', 'B_wnm_all', 'B_wnm_helix', 'C_final_seq', 'C_sap', 'C_score_per_res', 'C_total_score', 'C_wnm_all', 'C_wnm_helix', 'Y_cms', 'Y_ddg', 'Y_mean_pae', 'Y_mean_pae_interaction', 'Y_mean_pae_interaction_AB', 'Y_mean_pae_interaction_BA', 'Y_mean_pae_intra_chain', 'Y_mean_pae_intra_chain_A', 'Y_mean_pae_intra_chain_B', 'Y_mean_plddt', 'Y_model', 'Y_pTMscore', 'Y_recycles', 'Y_rmsd_to_reference', 'Y_sap', 'Y_score_per_res', 'Y_seed', 'Y_tol', 'Y_total_score', 'Y_type', 'bb_clash', 'bb_clash_delta', 'bb_clash_delta_x', 'best_average_DAN_plddts', 'best_average_plddts', 'best_model', 'best_ptm', 'best_rmsd_to_input', 'buns_parent', 'cms_AcB', 'cms_AnAc', 'cms_AnAcB', 'cms_AnB', 'designed_by', 'docked_helix', 'dssp', 'elapsed_time', 'exposed_hydrophobics_parent', 'geometry_parent', 'holes_all_parent', 'holes_core_parent', 'interface', 'loop_dist', 

### Add data from experiments

In [12]:
good_sec = {name: False for name in sample_df.index}
binds = {name: False for name in sample_df.index}
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0007/09_filter_and_order_739f0d7fe9bd45f986942789620a6f94.pdb.bz2"
] = True
binds[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0007/09_filter_and_order_739f0d7fe9bd45f986942789620a6f94.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_d07a25241ccd4c968d9c959e563c6ecb.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_b06794938c094af0a9e8bf480b919640.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_b70f3cd8223848728effbcf15defc053.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0001/09_filter_and_order_1cde2a1cdf5a43b89e435a5525be00f7.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_c83240588a174ccebcfe3fb61d1660b1.pdb.bz2"
] = True
binds[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_c83240588a174ccebcfe3fb61d1660b1.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0006/09_filter_and_order_d5f2693859a149b9b27f34e24e577819.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0006/09_filter_and_order_0b592d906d144108a845e9288a3be6b6.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_e843aaa59cf14fadbbf752bb07a548b0.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_1e3b2aab29414c85acd4b5738d4ba4b1.pdb.bz2"
] = True
binds[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_1e3b2aab29414c85acd4b5738d4ba4b1.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_81621ef5e5204b6ca56877a661b9bf63.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_8f66e3c2aa414e35add1c6572b7f3609.pdb.bz2"
] = True
good_sec[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_b5e055c110be4c49b4056a2b29ce3e09.pdb.bz2"
] = True
binds[
    "/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_b5e055c110be4c49b4056a2b29ce3e09.pdb.bz2"
] = True

sample_df

,A_final_seq,A_sap,A_score_per_res,A_total_score,A_wnm_all,A_wnm_helix,B_final_seq,B_sap,B_score_per_res,B_total_score,...,delta_sap_XY,delta_sap_AC,delta_total_score_XY,delta_total_score_AC,A_charge,B_charge,id,B_final_seq_r1,B_final_seq_r2,incorrect_id
pdb_path,,,,,,,,,,,,,,,,,,,,,
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0006/09_filter_and_order_e9b844167bf84dd2ac7cde5045dc43b1.pdb.bz2,DEEVQEAVERAEELREEAEELIKKARKTGDPELLRKALEALEEAVR...,70.064735,-1.903443,-437.791779,0.485354,0.086336,DEKEIKELAKRVAEKIREAFKIAAEGIKKGLEKK,40.034809,-1.143339,-38.873539,...,-3.854572,-31.276978,-517.192989,-96.144745,-12.039856,2.771885,cs_200,IKELAEEVAEKIREAFEIAAEGIEKGLEKR,DIKELAEEVAEKIREAFEIAAQGIEKGL,cs_219
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0007/09_filter_and_order_739f0d7fe9bd45f986942789620a6f94.pdb.bz2,DEEVQEAVERAEELREEAEELIKKARKTGDPELLRKALEALKEAVR...,57.506268,-2.112674,-485.915070,1.525385,0.076361,VEKEILELAKEVAEEIKEAIKIAAEGIRKGLEKK,49.365849,-1.043685,-35.485302,...,-9.132225,-20.905426,-482.052235,-24.410553,-14.027188,-1.247720,cs_201,SILELAEEVAEEIKEAIRIAAKGIREGLKE,SILELAEEVAEEIKEAIQIAAEGIRRGL,cs_220
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0007/09_filter_and_order_bd28e57868d440cdb4173b0c83f783f5.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLRELARALVEEAEAVA...,66.934334,-2.067249,-475.467133,0.617158,0.165406,EEEELLRELAELLVRVGRELLEMVEEQVKMIEERK,47.761921,-0.916358,-32.072525,...,-12.801592,-36.269396,-461.256271,-72.235199,-13.394478,-6.134295,cs_202,LLEELAELLVKVGRELLKMVQEQVQMIEER,LLEELAELLVEVGRELLKMVEEQVQMIK,cs_213
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_d07a25241ccd4c968d9c959e563c6ecb.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLEELAEALIEEAKAVA...,64.631165,-2.199711,-505.933533,0.491056,0.176779,AVREKLRELVELLVEVGRRLERVVRRAAERVKRRK,49.346237,-0.892982,-31.254385,...,-20.927460,-29.915047,-556.007822,-38.304382,-28.370214,6.809640,cs_203,VREKLRELVELLVEVGRRLEEVVKRAAERV,KLRELVELLVEVGRRLEEVVERAAERVE,cs_214
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_b06794938c094af0a9e8bf480b919640.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLEELAEALIEEAKAVA...,73.201729,-2.277482,-523.820801,0.348891,0.083955,AVREKLRELVELLVRVGERLRRVAERAAELVKRRR,41.399914,-0.937563,-32.814716,...,-11.018291,-30.554176,-594.738811,-18.317566,-22.385265,5.810639,cs_204,VEEKLRELVELLVHVGERLREVAERAAELV,KLRELVELLVEVGERLREVAERAAELVE,cs_215
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_b70f3cd8223848728effbcf15defc053.pdb.bz2,TEDERRELEKVARKAIEAAREGNTDEVREQLQRALEIARESGSEEA...,50.187321,-2.087024,-367.316223,0.295166,0.078919,MEEERKELTEKMLEVIDKLEKLVEEYYK,42.801014,-0.605469,-16.953135,...,-12.100096,-33.154501,-420.831411,-41.047974,-14.498245,-4.480233,cs_205,SEEEKEELTRKMLEVIDKLEQLVREYYK,SEEEKEELTRKMLEVIDKLEELVKKYYE,cs_207
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0001/09_filter_and_order_1cde2a1cdf5a43b89e435a5525be00f7.pdb.bz2,TEDERRELEKVARKAIEAAREGNTDEVREQLQRALEIARESGSEEA...,56.845699,-2.019813,-355.487030,0.329188,0.085122,SAKEREELTREFLEVIERLERLVREYYS,48.545830,-0.610745,-17.100849,...,-15.221077,-34.381081,-416.013622,-51.909607,-14.499890,-2.520417,cs_206,SEEEREELTKEFLEVIKRLEELVRRYYT,SEEERKELTEEFLEVIKRLEQLVREYYT,cs_208
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0002/09_filter_and_order_c83240588a174ccebcfe3fb61d1660b1.pdb.bz2,TEDERRELEKVARKAIEAAREGNTDEVREQLQRALEIARESGSEEA...,65.113930,-2.180363,-383.743866,0.356810,0.086168,MEEARKELTREMIEVLREIEEGFKERFE,33.392342,-0.619831,-17.355267,...,-0.200184,-32.033207,-443.054768,-23.099518,-7.514810,-4.468341,cs_207,PDEARKELTEEMIRVLEEIERGFKERFE,PEEARKELTEEMIKVLEEIERGFKERFE,cs_209
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0006/09_filter_

In [13]:
sec_df = pd.DataFrame.from_dict(good_sec, orient="index", columns=["sec"])
bind_df = pd.DataFrame.from_dict(binds, orient="index", columns=["binds"])
sample_df = pd.concat([sample_df, sec_df, bind_df], axis=1)
sample_df.head()

,A_final_seq,A_sap,A_score_per_res,A_total_score,A_wnm_all,A_wnm_helix,B_final_seq,B_sap,B_score_per_res,B_total_score,...,delta_total_score_XY,delta_total_score_AC,A_charge,B_charge,id,B_final_seq_r1,B_final_seq_r2,incorrect_id,sec,binds
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0006/09_filter_and_order_e9b844167bf84dd2ac7cde5045dc43b1.pdb.bz2,DEEVQEAVERAEELREEAEELIKKARKTGDPELLRKALEALEEAVR...,70.064735,-1.903443,-437.791779,0.485354,0.086336,DEKEIKELAKRVAEKIREAFKIAAEGIKKGLEKK,40.034809,-1.143339,-38.873539,...,-517.192989,-96.144745,-12.039856,2.771885,cs_200,IKELAEEVAEKIREAFEIAAEGIEKGLEKR,DIKELAEEVAEKIREAFEIAAQGIEKGL,cs_219,False,False
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0007/09_filter_and_order_739f0d7fe9bd45f986942789620a6f94.pdb.bz2,DEEVQEAVERAEELREEAEELIKKARKTGDPELLRKALEALKEAVR...,57.506268,-2.112674,-485.915070,1.525385,0.076361,VEKEILELAKEVAEEIKEAIKIAAEGIRKGLEKK,49.365849,-1.043685,-35.485302,...,-482.052235,-24.410553,-14.027188,-1.247720,cs_201,SILELAEEVAEEIKEAIRIAAKGIREGLKE,SILELAEEVAEEIKEAIQIAAEGIRRGL,cs_220,True,True
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0007/09_filter_and_order_bd28e57868d440cdb4173b0c83f783f5.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLRELARALVEEAEAVA...,66.934334,-2.067249,-475.467133,0.617158,0.165406,EEEELLRELAELLVRVGRELLEMVEEQVKMIEERK,47.761921,-0.916358,-32.072525,...,-461.256271,-72.235199,-13.394478,-6.134295,cs_202,LLEELAELLVKVGRELLKMVQEQVQMIEER,LLEELAELLVEVGRELLKMVEEQVQMIK,cs_213,False,False
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_d07a25241ccd4c968d9c959e563c6ecb.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLEELAEALIEEAKAVA...,64.631165,-2.199711,-505.933533,0.491056,0.176779,AVREKLRELVELLVEVGRRLERVVRRAAERVKRRK,49.346237,-0.892982,-31.254385,...,-556.007822,-38.304382,-28.370214,6.809640,cs_203,VREKLRELVELLVEVGRRLEEVVKRAAERV,KLRELVELLVEVGRRLEEVVERAAERVE,cs_214,True,False
/mnt/projects/crispy_shifty/projects/crispy_shifties/09_filter_and_order/decoys/0003/09_filter_and_order_b06794938c094af0a9e8bf480b919640.pdb.bz2,SEELERESEEAERRLQEARKRSEEARERGDLEELAEALIEEAKAVA...,73.201729,-2.277482,-523.820801,0.348891,0.083955,AVREKLRELVELLVRVGERLRRVAERAAELVKRRR,41.399914,-0.937563,-32.814716,...,-594.738811,-18.317566,-22.385265,5.810639,cs_204,VEEKLRELVELLVHVGERLREVAERAAELV,KLRELVELLVEVGERLREVAERAAELVE,cs_215,True,False


In [14]:
from crispy_shifty.utils.plotting import histplot_df, pairplot_df

In [24]:
print(
    {
        "DHR1.pdb",
        "DHR10.pdb",
        "DHR14.pdb",
        "DHR15.pdb",
        "DHR18.pdb",
        "DHR20.pdb",
        "DHR21.pdb",
        "DHR23.pdb",
        "DHR24.pdb",
        "DHR26.pdb",
        "DHR27.pdb",
        "DHR3.pdb",
        "DHR31.pdb",
        "DHR32.pdb",
        "DHR36.pdb",
        "DHR39.pdb",
        "DHR4.pdb",
        "DHR46.pdb",
        "DHR47.pdb",
        "DHR49.pdb",
        "DHR5.pdb",
        "DHR52.pdb",
        "DHR53.pdb",
        "DHR54.pdb",
        "DHR55.pdb",
        "DHR57.pdb",
        "DHR58.pdb",
        "DHR59.pdb",
        "DHR62.pdb",
        "DHR64.pdb",
        "DHR68.pdb",
        "DHR7.pdb",
        "DHR70.pdb",
        "DHR71.pdb",
        "DHR72.pdb",
        "DHR76.pdb",
        "DHR77.pdb",
        "DHR78.pdb",
        "DHR79.pdb",
        "DHR8.pdb",
        "DHR80.pdb",
        "DHR81.pdb",
        "DHR82.pdb",
        "DHR9.pdb",
    }
    - set([x + ".pdb" for x in scores_df.parent.values if "DHR" in x])
)

{'DHR5.pdb', 'DHR78.pdb', 'DHR8.pdb', 'DHR58.pdb', 'DHR15.pdb', 'DHR3.pdb', 'DHR36.pdb', 'DHR77.pdb', 'DHR72.pdb', 'DHR9.pdb', 'DHR57.pdb', 'DHR55.pdb', 'DHR18.pdb', 'DHR10.pdb', 'DHR39.pdb', 'DHR53.pdb', 'DHR1.pdb', 'DHR23.pdb', 'DHR31.pdb', 'DHR4.pdb'}


### TODO

In [8]:
from crispy_shifty.utils.design import beta_nov16_terms

sample_df = sample_df[
    [term for term in sample_df.columns if term not in beta_nov16_terms]
]
print(len(sample_df))
print(list(sample_df.columns))

1031
['A_final_seq', 'A_sap', 'A_score_per_res', 'A_total_score', 'A_wnm_all', 'A_wnm_helix', 'B_final_seq', 'B_sap', 'B_score_per_res', 'B_total_score', 'B_wnm_all', 'B_wnm_helix', 'C_final_seq', 'C_sap', 'C_score_per_res', 'C_total_score', 'C_wnm_all', 'C_wnm_helix', 'Y_cms', 'Y_ddg', 'Y_mean_pae', 'Y_mean_pae_interaction', 'Y_mean_pae_interaction_AB', 'Y_mean_pae_interaction_BA', 'Y_mean_pae_intra_chain', 'Y_mean_pae_intra_chain_A', 'Y_mean_pae_intra_chain_B', 'Y_mean_plddt', 'Y_model', 'Y_pTMscore', 'Y_recycles', 'Y_rmsd_to_reference', 'Y_sap', 'Y_score_per_res', 'Y_seed', 'Y_tol', 'Y_total_score', 'Y_type', 'bb_clash', 'bb_clash_delta', 'bb_clash_delta_x', 'best_average_DAN_plddts', 'best_average_plddts', 'best_model', 'best_ptm', 'best_rmsd_to_input', 'buns_parent', 'cms_AcB', 'cms_AnAc', 'cms_AnAcB', 'cms_AnB', 'designed_by', 'docked_helix', 'dssp', 'elapsed_time', 'exposed_hydrophobics_parent', 'geometry_parent', 'holes_all_parent', 'holes_core_parent', 'interface', 'loop_dist'

### Save a list of outputs

In [10]:
# output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")

# with open(os.path.join(output_path, "filtered_states.list"), "w") as f:
#     for path in tqdm(scores_df.index):
#         print(path, file=f)

  0%|          | 0/48561 [00:00<?, ?it/s]

### Prototyping blocks

test `finalize_peptide`

In [ ]:
%%time 
import pyrosetta

pyrosetta.init(
    "-corrections::beta_nov16  -mute all"
)


sys.path.insert(0, "/projects/crispy_shifty/")
from crispy_shifty.protocols.cleaning import finalize_peptide

t = finalize_peptide(
        None,
        **{
            'pdb_path': '/mnt/projects/crispy_shifty/projects/crispy_shifties/08_fold_paired_states_X/decoys/0052/08_fold_paired_states_X_d9d44e6fa15b4dd39296e2f1d3055a12.pdb.bz2',
            'yeet_chain': '3',
            'models': '1',
        }
)
for i, tppose in enumerate(t):
    tppose.pose.dump_pdb(f"{i}.pdb")

test `filter_paired_state`

In [ ]:
# %%time
# import pyrosetta

# pyrosetta.init(
# "-corrections::beta_nov16 -indexed_structure_store:fragment_store /net/databases/VALL_clustered/connect_chains/ss_grouped_vall_helix_shortLoop.h5"
# )


# sys.path.insert(0, "/projects/crispy_shifty/")
# from crispy_shifty.protocols.msd import filter_paired_state

# t = filter_paired_state(
#         None,
#         **{
#             'pdb_path': '/mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_760c687ae53646b59bd36bbe91998593.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_2118b14f45e849ac99034c2e36a8230f.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_806db75a0f8949119a54d50ee8c28576.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_bf4fc1710acb4037917254ce283a7223.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_1984895bd0d744e7b7c82446e07b8189.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_ff38fa4457a84da1b0d95850f99b6f10.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_c712e867dda84adab431812f7d1ed004.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_5b83ffb2312944f084a5557ed82ed5d9.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_b319b9e9da43467596e807f54262cde2.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_fecc3fb836b24903af7b13df546ea87c.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_1d931fe681f0482098275e4fd0749d3e.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_e942561def5b4e4491b3c9d9b2d15f76.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_b4791741764a4a1eb28a624286d32a12.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_261a0384260a4d169bab95c703574b15.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_5ff0e503a5704aa39ed3b721793432e5.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_ea30c907f4d54a0a807e2b3475004032.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_a62763836a9a4543858b45840ede8c70.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_d2efc01301a74800b6f5669dba46434b.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_b60157d157614d5eb8d540ec1cad6cbd.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_d66fa964e5d545feaa20bd131bb372d5.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_c3e8d46a19c3441fb1c02e6ab8cc0f1e.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_f763ddc8862f4716ab73613645b818b5.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_b5c33cfd51c74dfa8259136db8fc02e9.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_00e8a2bb6e3241a78f3688a5727f53ca.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_1e151d604abe47aca2d3b0d2ca8449bd.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_dda3a86db52549d5881ea816b39fe2fa.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_497eff9bccbc466999ac1445d8c72307.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_35a59b20c8a342dab46b094dc085fbbe.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_9e1e91449de0489a95ecd922947c130c.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_45c296059f6a45879dd55ddb1909bdfd.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_9a8528df5cdd405d81a1d825e281a7d8.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_23965c49bce44dc88f1d373832948c11.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_45dce59a18a742b28c5ade3172290005.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_c6f1549c40824fe4a7b71e548ae44237.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_96b04c0f1f834a07a0de2240712e91dd.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_2e6f7255b1344097b82f1ce9d29b9117.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_0c915ad2756c4728ae0e5292ae9aa73e.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_018f599f71d649bda48372a0f63a6337.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_892f31cdce9e4265b491359fc274d050.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_ab90cfa6b99f4df792b36c5afc737631.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_e8d8ae2c9c684a1c927ca22d0cfee18f.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_a2998e9517124945844771da288f5ad0.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_335f541ac8084aaa99ecae31bada0095.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_bd0143a910f14248a19c9c8b4d05a551.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_9bd90f2f947642ec8e45dc9aa29cd095.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_bd53adc3fba34f378958fb65820ff1c5.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_7dfa893d4bee4a4a961c70f127e33c13.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_d44931e5caf743bab11932b80e2659ff.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_63921631bf6e44d19d5c978877f830f7.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_25f37915e4d74cef85291bc417db4118.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_12f06b9607dc4f19b6bd12e99f4514ec.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_28182b7490bb430eae7ccde30a0d2754.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_d9c4a62d284c44258cfa185e6cffddcb.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_5b58abbc36184d88b4039d156541ad04.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_90772bccd98e42bc821ffc28da0aa225.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_f4a09a7f494f49fea4c1535328dc615d.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_c8a9c5ba7e9b490ca72b98e86182cc22.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_a06efe01bba64d39992b001faf05be0c.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_d313e754e58a4005a3a1ab885ac2efbb.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_6061f155ab794b0b83f6f2e19db15473.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_3f2369ac5c0c471b9e686b782f761244.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_410e3919547f4adcaedcf972769a6950.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_56f46e89edfb4d83a2102e142f7c1f56.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_9f89cb16e6bf4b26b7f55722659e0596.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_26c2d2d3350d47e28ea86a272299987a.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_507d748832f242d79d589557ff8f8c3a.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_5ea6d6ffdf7b4d66b4a1d1fd5de60175.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_986cd31fdbfa47e4af3111b964db5e11.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_f78e47f8d1ea4bf59b53425b29726f96.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_5c820da32f73451d847e5e6a3abad86d.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_676b0769624245b9a34e6dfa1dee3ef5.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_c97be179d46f4ec08a34153b42b68b0c.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_981e6b2cb50f41f8aee5bda9c8a50c71.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_08c4e9c1de034b09a59add4f767fe65a.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_e1815ba370b64285ae5000b3d4b16292.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_70cf0471881146ac93054f9082a17639.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_5d84254a968e481dba4592d3e2b63dc6.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_1b757e1311cc4456a7c510042261493d.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_26f9b1b83be1413c87f681543221cac9.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_f76a614bd449489182e987ecb13a9ce0.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_7f3c8d9f96ba4ec68596a11422725cdd.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_a6429f90372e4cc8a85b415188914a55.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_403c1f10d975414e890a0b9b280eaf0a.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_a582f049287b4b598403c0fd82d217b2.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_1d427ef8b2cf43959c61847fef0c9e89.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_10a2d6c9762f4a54ba7c21aecae4b270.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_586163d0e4134e5dbe52efab1a0d67c3.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_4a79bf27aad4406a87f99b23dcbddce7.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_3861ea9bae014dc88c63763f909f0a54.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_8272675c92b84570a0386ad84498991b.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_68c8c93fc0444e4ca2e1fbf1151d944b.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_646b1f3e41244da5ae625cc8c7668de4.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_d086fd3d9e2c490faf45e8c60ff3fb93.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_e7f52917fe494a20a32d89860fd906b6.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_f9fd36225b9441fa92107490d388efb5.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_1da1cf24ec7a4e6ca139ab3dba7752c5.pdb.bz2 /mnt/home/pleung/projects/crispy_shifty/projects/crispy_shifties/07_fold_paired_states_Y/decoys/0000/07_fold_paired_states_Y_d34ed61477704b7eb443640b82c1cc3c.pdb.bz2',
#         }
# )
# for i, tppose in enumerate(t):
#     tppose.pose.dump_pdb(f"{i}.pdb")